In [ ]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import os
from carbonplan_forest_risks.utils import get_store, albers_ak_transform
import numpy as np
import warnings
from carbonplan_forest_risks.load import terraclim
import pandas as pd
warnings.filterwarnings('ignore')
account_key = os.environ.get('BLOB_ACCOUNT_KEY')

In [ ]:
gcms = [
    ("CanESM5-CanOE", "r3i1p2f1"),
    ("MIROC-ES2L", "r1i1p1f2"),
    ("ACCESS-CM2", "r1i1p1f1"),
    ("ACCESS-ESM1-5", "r10i1p1f1"),
    ("MRI-ESM2-0", "r1i1p1f1"),
    ("MPI-ESM1-2-LR", "r10i1p1f1"),
]

In [ ]:
v1_template = "cmip6/quantile-mapping/conus/4000m/monthly/{}.{}.{}.zarr"

v2_template = "cmip6/quantile-mapping-v2/conus/4000m/monthly/{}.{}.{}.zarr"

In [ ]:
account_key = os.environ.get("BLOB_ACCOUNT_KEY")

In [ ]:
models = []
for scenario in ["historical", "ssp245", "ssp370", "ssp585"]:
    for (gcm, ensemble_member) in gcms:
        models.append("{}-{}".format(gcm, scenario))
path = get_store(
    "carbonplan-downscaling",
    zarr_template.format(gcms[0][0], "historical", gcms[0][1]),
    account_key=account_key,
)
ds = xr.open_zarr(path)
df = pd.DataFrame(index=models, columns=ds.data_vars)

In [ ]:
for scenario in ["historical", "ssp245", "ssp370", "ssp585"]:
    for (gcm, ensemble_member) in gcms:
        path = get_store(
            "carbonplan-downscaling",
            zarr_template.format(gcm, scenario, ensemble_member),
            account_key=account_key,
        )
        ds = xr.open_zarr(path)
        for var in ds.data_vars:
            df.loc["{}-{}".format(gcm, scenario), var] = (
                ds[var].isnull().sum().values
            )

In [ ]:
ds_v1 = xr.open_zarr(
    get_store(
        "carbonplan-downscaling",
        zarr_template.format(gcms[0][0], "historical", gcms[0][1]),
        account_key=account_key,
    )
)

In [ ]:
ds_v1.pdsi.mean(dim="time", skipna=False).isnull().plot()

In [ ]:
ds2 = xr.open_zarr(
    get_store(
        "carbonplan-downscaling",
        v2_template.format("ACCESS-ESM1-5", "ssp245", "r10i1p1f1"),
        account_key=account_key,
    )
)

In [ ]:
ds2.vpd.mean(dim="time", skipna=False).plot()

In [ ]:
ds2.pdsi.isnull().sum().values

In [ ]:
ds_v1 = xr.open_zarr(
    get_store(
        "carbonplan-downscaling",
        v1_template.format("ACCESS-ESM1-5", "ssp245", "r10i1p1f1"),
        account_key=account_key,
    )
)

In [ ]:
ds_v1.vap.min(dim="time", skipna=False).plot()

In [ ]:
ds_v1.pdsi.isnull().sum().values

In [ ]:
ds3 = xr.open_zarr(
    get_store(
        "carbonplan-downscaling",
        zarr_template.format("MRI-ESM2-0", "historical", "r1i1p1f1"),
        account_key=account_key,
    )
)

In [ ]:
ds3.pdsi.mean(dim="time", skipna=False).isnull().plot()

In [ ]:
df[df.index.str.contains("historical")] -= 350691120
df[df.index.str.contains("ssp")] -= 463991328

In [ ]:
df

In [ ]:
wyoming = {"x": slice(-1.2e6, -1e6), "y": slice(2.6e6, 2.2e6)}

In [ ]:
(terraclimate_v3.sel(**wyoming).cwd.min(dim="time")).plot()

In [ ]:
ds3["def"].max(dim="time").plot(vmin=99, vmax=101)  # .vap==0).sum()

In [ ]:
test = load.terraclim(
    store="az",
    sampling="monthly",
    variables=["ppt", "tmean", "cwd", "vap", "vpd"],
)